In [5]:
filepath = './datasets'
processed_data = './processed_data'
gauge_path = filepath+'/gauge.csv'
radar_path = filepath+'/radar'

In [6]:
from tqdm import tqdm_notebook as tqdm
from scipy.stats import pearsonr

In [7]:
import pandas as pd
import numpy as np
import os

In [54]:
df_gauge = pd.read_csv(gauge_path)

In [55]:
df_result = pd.DataFrame()
train_week = []
for filename in tqdm(os.listdir(radar_path)):
    week = filename.split('.')[0][-2:]
    train_week.append(str(week))
    full_radar_path = os.path.join(radar_path, filename)
    df_radar = pd.read_csv(full_radar_path)
    radar_obs = df_radar.values[:,2:]
    radar_arange = np.arange(0, 34)
    radar_mean = np.matmul(radar_obs, radar_arange) / 2016.
    if 'latlong' not in df_result.columns:
        df_result['latlong'] = df_radar['lat'] * 1000 + df_radar['long']
    assert np.allclose(df_result['latlong'].values, df_radar['lat'] * 1000 + df_radar['long'])
    df_result['mean_week_'+str(week)] = radar_mean

HBox(children=(IntProgress(value=0, max=31), HTML(value='')))

In [56]:
train_week

['18',
 '36',
 '38',
 '31',
 '44',
 '16',
 '41',
 '21',
 '47',
 '33',
 '48',
 '17',
 '20',
 '51',
 '39',
 '35',
 '22',
 '15',
 '37',
 '29',
 '45',
 '13',
 '26',
 '19',
 '23',
 '49',
 '50',
 '46',
 '32',
 '24',
 '28']

In [57]:
df_result = df_result[sorted(df_result.columns)]

In [58]:
df_result.head()

,latlong,mean_week_13,mean_week_15,mean_week_16,mean_week_17,mean_week_18,mean_week_19,mean_week_20,mean_week_21,mean_week_22,...,mean_week_39,mean_week_41,mean_week_44,mean_week_45,mean_week_46,mean_week_47,mean_week_48,mean_week_49,mean_week_50,mean_week_51
0,0,0.675099,0.382937,0.535714,0.815476,0.248016,0.237599,0.416667,0.908730,0.306052,...,0.193948,0.994544,0.695933,1.006944,1.025298,0.670139,0.461310,0.645337,0.406250,0.157242
1,1,0.714782,0.382440,0.493552,0.841270,0.258433,0.266369,0.450397,0.963790,0.289187,...,0.164683,0.980655,0.738591,1.010417,1.021825,0.635913,0.457341,0.637897,0.339782,0.183532
2,2,0.742063,0.411706,0.519345,0.859127,0.235615,0.260417,0.466270,0.968254,0.324901,...,0.145833,0.958333,0.691468,1.002480,1.012897,0.616071,0.469742,0.611607,0.336806,0.154266
3,3,0.746032,0.380456,0.522321,0.887897,0.247520,0.278274,0.474702,0.954365,0.307540,...,0.157738,0.932044,0.755952,1.075893,1.037202,0.561012,0.494048,0.574405,0.350694,0.157242
4,4,0.745536,0.380456,0.519345,0.860119,0.248016,0.277778,0.472222,0.950893,0.304563,...,0.155258,0.923115,0.771825,1.070933,1.038690,0.553075,0.499008,0.573909,0.352183,0.154266


In [59]:
df_gauge = pd.read_csv(gauge_path)

In [60]:
len(np.where(pd.isnull(df_gauge))[0])

357

In [61]:
df_gauge['Year'] = df_gauge['Year'].astype(str)
df_gauge['Week'] = df_gauge['Week'].astype(str)
df_gauge = df_gauge.apply(lambda x: x.fillna(x[2:].mean()),axis=1)

In [62]:
train_week = sorted(train_week)

In [63]:
train_bool = df_gauge['Week'].apply(lambda x : x in train_week)
train_df_gauge = df_gauge[train_bool]

In [64]:
train_df_gauge_array = train_df_gauge.values[:, 2:]

In [65]:
train_df_gauge.head()

,Year,Week,0,1,2,3,4,5,6,7,...,40,41,42,43,44,45,46,47,48,49
12,2017,13,36.8,30.0,48.2,78.6,67.4,45.6,77.6,21.6,...,37.800000,34.0,43.645455,19.4,25.2,26.6,4.8,57.000000,107.000000,54.000000
14,2017,15,43.8,90.4,82.6,13.2,45.8,86.6,34.6,38.4,...,46.000000,99.0,66.773810,66.6,72.9,76.2,82.4,53.600000,21.800000,66.773810
15,2017,16,98.8,134.0,68.2,74.8,77.7,86.4,37.8,102.6,...,97.579487,139.0,97.579487,58.6,69.7,68.2,162.6,97.579487,43.000000,97.579487
16,2017,17,100.2,56.0,82.8,69.2,127.3,84.6,78.0,24.8,...,64.136842,46.2,89.200000,28.0,38.7,112.6,17.8,34.600000,64.136842,32.400000
17,2017,18,2.0,76.0,1.6,1.2,54.5,25.2,30.4,49.8,...,36.884211,24.8,36.884211,29.2,25.5,109.8,71.6,14.000000,36.884211,9.600000


In [66]:
train_df_gauge_array.shape

(31, 50)

In [67]:
df_result['latlong'].values

array([     0,      1,      2, ..., 479477, 479478, 479479])

In [69]:
print (np.where(pd.isnull(train_df_gauge_array)))
print (np.where(pd.isnull(df_result)))

(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))


In [ ]:
df_corr_result = pd.DataFrame()
df_corr_result['latlong'] = df_result['latlong'].values
for station in tqdm(range(train_df_gauge_array.shape[1])):
    station_values = train_df_gauge_array[:, station]
    total_latlong = []
    for idx, latlong in tqdm(df_result.iterrows()):
        latlong_values = latlong.values[1:]
        correlation = np.corrcoef(station_values.tolist(), latlong_values.tolist())[0][1]
        total_latlong.append(correlation)
    df_corr_result['corr_station_'+str(station)] = total_latlong

In [131]:
df_corr_result = pd.read_csv('toto_corr_45.csv')
df_corr_result = df_corr_result.fillna(0.0)

In [132]:
df_corr_result.sort_values('corr_station_49', ascending = False).head()

,lat,long,corr_station_0,corr_station_1,corr_station_2,corr_station_3,corr_station_4,corr_station_5,corr_station_6,corr_station_7,...,corr_station_42,corr_station_43,corr_station_44,corr_station_45,corr_station_46,corr_station_47,corr_station_48,corr_station_49,x,y
19128,1.4626,103.6506,0.341151,0.386757,0.212099,0.186730,0.173509,0.436805,0.091700,0.211649,...,0.292993,0.393900,0.380867,0.202563,0.471756,0.459866,0.230946,0.741116,197,119
18230,1.4495,103.6532,0.312487,0.489154,0.300940,0.132599,0.130493,0.359512,0.143375,0.118871,...,0.399920,0.416286,0.436772,0.163896,0.417021,0.301902,0.202850,0.739824,202,120
18979,1.4600,103.6480,0.321431,0.376741,0.228763,0.149438,0.146362,0.399938,0.063213,0.156051,...,0.297337,0.398730,0.397497,0.193811,0.452855,0.395733,0.192498,0.734884,198,118
18674,1.4548,103.6506,0.286155,0.450447,0.212053,0.167323,0.096221,0.324213,0.082152,0.126494,...,0.339285,0.403489,0.430749,0.093744,0.407837,0.271155,0.196749,0.731267,200,119
18676,1.4548,103.6558,0.232719,0.385105,0.176996,0.090617,0.072255,0.382798,0.067071,0.045307,...,0.254111,0.363142,0.375104,0.119945,0.340763,0.246210,0.189247,0.726995,200,121


In [133]:
len(np.where(pd.isnull(df_corr_result))[0])

0

In [134]:
lat = df_corr_result.values[:,0]
long = df_corr_result.values[:, 1]
x = df_corr_result.values[:,52]
y = df_corr_result.values[:,53]

In [135]:
x.shape

(20223,)

In [136]:
df_corr_result.columns[2:52]

Index(['corr_station_0', 'corr_station_1', 'corr_station_2', 'corr_station_3',
       'corr_station_4', 'corr_station_5', 'corr_station_6', 'corr_station_7',
       'corr_station_8', 'corr_station_9', 'corr_station_10',
       'corr_station_11', 'corr_station_12', 'corr_station_13',
       'corr_station_14', 'corr_station_15', 'corr_station_16',
       'corr_station_17', 'corr_station_18', 'corr_station_19',
       'corr_station_20', 'corr_station_21', 'corr_station_22',
       'corr_station_23', 'corr_station_24', 'corr_station_25',
       'corr_station_26', 'corr_station_27', 'corr_station_28',
       'corr_station_29', 'corr_station_30', 'corr_station_31',
       'corr_station_32', 'corr_station_33', 'corr_station_34',
       'corr_station_35', 'corr_station_36', 'corr_station_37',
       'corr_station_38', 'corr_station_39', 'corr_station_40',
       'corr_station_41', 'corr_station_42', 'corr_station_43',
       'corr_station_44', 'corr_station_45', 'corr_station_46',
       'corr

In [137]:
df_corr_result_array = df_corr_result.values[:,2:52]
df_corr_result_array.shape

(20223, 50)

In [138]:
from GoogleMap import is_in_sg

In [139]:
dict_result = {}
for station in range(df_corr_result_array.shape[1]):
    values = df_corr_result_array[:, station]
    index = values.argsort()[::-1]
    for chosen_index in index:
        if is_in_sg(lat[chosen_index], long[chosen_index]):  
            dict_result[str(station)] = {
            'lat' : lat[chosen_index],
            'long': long[chosen_index],
            'x': x[chosen_index],
            'y': y[chosen_index]
            }
            break

In [140]:
import json
with open('station_location.json_new', 'w') as outfile:
    json.dump(dict_result, outfile)

In [141]:
dict_result

{'0': {'lat': 1.347, 'long': 103.9264, 'x': 241.0, 'y': 224.0},
 '1': {'lat': 1.4075, 'long': 103.6952, 'x': 218.0, 'y': 136.0},
 '10': {'lat': 1.4233, 'long': 103.6742, 'x': 212.0, 'y': 128.0},
 '11': {'lat': 1.2946, 'long': 103.795, 'x': 261.0, 'y': 174.0},
 '12': {'lat': 1.3418, 'long': 103.9316, 'x': 243.0, 'y': 226.0},
 '13': {'lat': 1.2972, 'long': 103.8528, 'x': 260.0, 'y': 196.0},
 '14': {'lat': 1.3155, 'long': 103.8686, 'x': 253.0, 'y': 202.0},
 '15': {'lat': 1.2263, 'long': 103.7399, 'x': 287.0, 'y': 153.0},
 '16': {'lat': 1.3155, 'long': 103.8686, 'x': 253.0, 'y': 202.0},
 '17': {'lat': 1.3839, 'long': 103.6611, 'x': 227.0, 'y': 123.0},
 '18': {'lat': 1.263, 'long': 103.8134, 'x': 273.0, 'y': 181.0},
 '19': {'lat': 1.2289, 'long': 103.7399, 'x': 286.0, 'y': 153.0},
 '2': {'lat': 1.3418, 'long': 103.8896, 'x': 243.0, 'y': 210.0},
 '20': {'lat': 1.3891, 'long': 103.7399, 'x': 225.0, 'y': 153.0},
 '21': {'lat': 1.1974, 'long': 103.8791, 'x': 298.0, 'y': 206.0},
 '22': {'lat': 1